Analysing the CSV file Peter got.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [2]:
soybean_yield = pd.read_csv('../noahdata/state soybeans.csv',thousands=',')

In [3]:
fertilizer =  pd.read_csv('../noahdata/SOYBEANS-Fertilizers-2019-11-13.csv',thousands=',')

In [4]:
import sys
sys.path.append('../useful_functions/')

In [5]:
import data_merger

In [6]:
data_merger.format_fertilizer(fertilizer)

Hopefully only 3 fertilizer types? If not fix this f.  4


../useful_functions/data_merger.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g[ftype + ' ' + col] = g[col]
/Users/noahkasmanoff/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,YEAR,LOCATION,NITROGEN APPLICATIONS in LB,"NITROGEN APPLICATIONS in LB / ACRE / APPLICATION, AVG","NITROGEN APPLICATIONS in LB / ACRE / YEAR, AVG","NITROGEN APPLICATIONS in NUMBER, AVG","NITROGEN TREATED in PCT OF AREA PLANTED, AVG",PHOSPHATE APPLICATIONS in LB,"PHOSPHATE APPLICATIONS in LB / ACRE / APPLICATION, AVG","PHOSPHATE APPLICATIONS in LB / ACRE / YEAR, AVG","PHOSPHATE APPLICATIONS in NUMBER, AVG","PHOSPHATE TREATED in PCT OF AREA PLANTED, AVG",POTASH APPLICATIONS in LB,"POTASH APPLICATIONS in LB / ACRE / APPLICATION, AVG","POTASH APPLICATIONS in LB / ACRE / YEAR, AVG","POTASH APPLICATIONS in NUMBER, AVG","POTASH TREATED in PCT OF AREA PLANTED, AVG"
0,1999,ILLINOIS,16200000,18,20,1.1,7,64100000,45,45,1.0,14,304000000,102,103,1.0,28
1,1999,IOWA,23500000,27,30,1.1,7,103500000,55,55,1.0,17,173700000,72,72,1.0,22
2,1999,MINNESOTA,18700000,20,20,1.0,13,29500000,33,33,1.0,13,54500000,58,58,1.0,13
3,2000,ILLINOIS,16800000,14,15,1.0,11,77500000,47,47,1.0,16,286000000,93,95,1.0,29
4,2000,IOWA,81000000,49,49,1.0,15,110100000,47,47,1.0,22,138000000,59,59,1.0,22
5,2000,MINNESOTA,10200000,16,17,1.0,8,24100000,34,35,1.0,9,118600000,67,69,1.0,24
6,2001,ILLINOIS,42800000,37,40,1.0,10,95800000,72,75,1.0,12,250500000,105,105,1.0,22
7,2001,IOWA,9900000,16,17,1.1,5,47900000,45,48,1.0,9,71300000,66,66,1.0,10
8,2001,MINNESOTA,15300000,16,16,1.0,13,32300000,34,34,1.0,13,41500000,48,48,1.0,12
9,2002,ILLINOIS,37500000,20,20,1.0,18,143100000,54,54,1.0,25,422600000,105,105,1.0,38


In [ ]:
fertilizer.groupby(["LOCATION","DOMAINCAT","YEAR"])['APPLICATIONS in LB'].value_counts().to_frame()

One idea now is to expand these columns so it goes year state fertilizer application 

In [ ]:
fertilizers = []


for ftype, g in fertilizer.groupby("DOMAINCAT"):
    ftype = ftype.split('(')[1].split(')')[0]
    for col in g.columns[8:]:
        g[ftype + ' ' + col] = g[col]
        del g[col]
    g.drop(columns= ['STATE ANSI', 'ASD CODE', 'COUNTY ANSI',
       'REFERENCE PERIOD', 'COMMODITY','DOMAINCAT'],inplace=True)
    fertilizers.append(g)


In [ ]:
fertilizer_df = pd.merge(fertilizers[0],fertilizers[1],on=['YEAR','LOCATION'])

In [ ]:
fertilizer_df = fertilizer_df.merge(fertilizers[2],on = ['YEAR','LOCATION'])

In [ ]:
fertilizer_df.columns

# Now try and merge fertilizer df with soybeans!

In [ ]:
state_soybean_yield = soybean_yield.loc[soybean_yield['REFERENCE PERIOD'] == 'YEAR']

In [ ]:
state_yields = state_soybean_yield.merge(fertilizer_df,on = ['YEAR',
                                             'LOCATION'])

In [ ]:
state_yields.columns

In [ ]:
X = state_yields[['NITROGEN APPLICATIONS in LB',
       'NITROGEN APPLICATIONS in LB / ACRE / APPLICATION, AVG',
       'NITROGEN APPLICATIONS in LB / ACRE / YEAR, AVG',
       'NITROGEN APPLICATIONS in NUMBER, AVG',
       'NITROGEN TREATED in PCT OF AREA PLANTED, AVG',
       'PHOSPHATE APPLICATIONS in LB',
       'PHOSPHATE APPLICATIONS in LB / ACRE / APPLICATION, AVG',
       'PHOSPHATE APPLICATIONS in LB / ACRE / YEAR, AVG',
       'PHOSPHATE APPLICATIONS in NUMBER, AVG',
       'PHOSPHATE TREATED in PCT OF AREA PLANTED, AVG',
       'POTASH APPLICATIONS in LB',
       'POTASH APPLICATIONS in LB / ACRE / APPLICATION, AVG',
       'POTASH APPLICATIONS in LB / ACRE / YEAR, AVG',
       'POTASH APPLICATIONS in NUMBER, AVG',
       'POTASH TREATED in PCT OF AREA PLANTED, AVG']]
X = X.astype(float)
X_df = X.copy(deep = True)
y = state_yields['YIELD in BU / ACRE']
y = y.astype(float)

In [ ]:
# must normalize first! 

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X,y)

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linreg = LinearRegression()

In [ ]:
linreg.fit(X_train,y_train)

In [ ]:
linreg.score(X_test,y_test)

So that is our baseline RSS score... not great, not terrible. 


Hopefully someone reading this has watched Chernobyl. 


Now below I'll add in a few other EDA steps, and hope to add more or derive better features later on. 

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(X_df.corr())

What's interesting about this grid is the anti-correlation between different fertilizer types... hmmmmm 